## Bacterial growth curves

This notebook reproduces the analysis to create bacterial growth curves. Images are first aligned and segmented. Then the cells are tracked from frame to frame using the linear assignment problem approach.

First we have to import packages

In [1]:
import h5py
import tifffile as tiff
from keras.backend.common import _UID_PREFIXES

from cnn_functions import nikon_getfiles, get_image, run_models_on_directory, get_image_sizes, segment_nuclei, segment_cytoplasm, dice_jaccard_indices
from model_zoo import sparse_bn_feature_net_31x31 as cyto_fn

import os
import numpy as np

from cnn_functions import get_image, align_images, crop_images, make_tracks, get_lineage
from cnn_functions import create_masks, plot_lineage, plot_lineage_numbers, plot_lineage_total
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
import tifffile as tiff

mpl.rcParams['pdf.fonttype'] = 42


Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5005)


Let's define the directories that we'll need to proceed.

In [2]:
direc_name = "/home/vanvalen/Data/ecoli"
image_dir = os.path.join(direc_name, "RawImages/")
align_dir = os.path.join(direc_name, "Align/")
cnn_save_dir = os.path.join(direc_name, "Cytoplasm/")
mask_dir = os.path.join(direc_name, "Masks/")
region_dir = os.path.join(direc_name, "Regions/")
cropped_dir = os.path.join(direc_name, "Cropped/")
track_dir = os.path.join(direc_name, "Tracks/")
cost_dir = os.path.join(direc_name, "Cost_Matrices/")

Lets also define the phase channel (recall that we only use the phase images to segment bacteria)

In [3]:
channel_names = ["Pos3"]

First, we need to align our images

In [ ]:
align_images(direc_name = image_dir, channel_names = channel_names, direc_save = align_dir)

In [ ]:
trained_network_cyto_directory = "/home/vanvalen/DeepCell2/trained_networks/ecoli/ecoli_all"
cyto_prefix = "2016-07-20_ecoli_all_31x31_bn_feature_net_31x31_"
win_cyto = 15
win_nuclear = 15

image_size_x, image_size_y = get_image_sizes(align_dir, channel_names)

list_of_cyto_weights = []
for j in xrange(5):
	cyto_weights = os.path.join(trained_network_cyto_directory,  cyto_prefix + str(j) + ".h5")
	list_of_cyto_weights += [cyto_weights]
    
cytoplasm_predictions = run_models_on_directory(align_dir, channel_names, cnn_save_dir, model_fn = cyto_fn, 
	list_of_weights = list_of_cyto_weights, image_size_x = image_size_x, image_size_y = image_size_y, 
	win_x = win_cyto, win_y = win_cyto, std = False, split = False)

Next we need to create masks from the segmentation prediction

In [4]:
create_masks(direc_name = cnn_save_dir, direc_save_mask = mask_dir, direc_save_region = region_dir, area_threshold = 25, clear_borders = 1)

Now we need to construct cell lineages. This is done with the following section of code.

In [ ]:
# Load Regions
region_file = np.load(os.path.join(region_dir, 'regions_save.npz'))
regions_save = region_file['regions_save']

total_no_of_frames = 32

# Construct tracks
list_of_tracks = []
for chunk in xrange(1,len(regions_save)):
	tracks = make_tracks(regions = regions_save[chunk], direc_save = track_dir, start_frame = 14, end_frame = 31, direc_cost_save = cost_dir)
	list_of_tracks += [tracks]

file_name_save = 'list_of_tracks'
np.savez(os.path.join(track_dir, file_name_save), tracks = list_of_tracks)

We can make single cell growth curves. First lets load the cell lineages we constructed.

In [ ]:
# Load Regions
region_file = np.load(region_dir + 'regions_save.npz')
regions = region_file['regions_save']

# Load Tracks
track_file = np.load(track_dir + 'list_of_tracks.npz')
list_of_tracks = track_file['tracks']

# Load Tracks
track_file = np.load(track_dir + 'tracks.npz')
tracks = track_file['tracks']
for track_num in xrange(len(tracks)):
    list_of_cells, lineage_ids = get_lineage(list_of_tracks[0],track_num)
    plot_lineage_numbers(list_of_cells, tracks, (870,870))

8 1
18 1


NameError: global name 'cf' is not defined

NameError: global name 'cf' is not defined

30 1
1 1
30 1


NameError: global name 'cf' is not defined

...

NameError: global name 'cf' is not defined

NameError: global name 'cf' is not defined

NameError: global name 'cf' is not defined

NameError: global name 'cf' is not defined

6 6
9 7
29 10
30 10
30 11
30 12
17 12
30 12
22 14
30 14
30 15
17 16
30 17


Now lets make the growth curves.

In [7]:
counter =  0
colors = [(0,0,0), (0.902,0.624,0),(0.337,0.706,0.914),(0,0.620,0.451),(0,0.447,0.698),(0.8,0.475,0.655)]
colors += colors
for chunk in list_of_tracks:
	for track in chunk:
		if len(track) > 12:
			if counter < 12:
				frame = [(temp['frame']-15)*5 for temp in track]
				area = [temp['area']*(.13 **2) for temp in track]
				
				plt.plot(np.array(frame)-frame[0],area, color = colors[counter])
				plt.xlabel('Time (min)', fontsize = 16)
				plt.ylabel(r'Area ($\mu m^2$)', fontsize = 16)
				plt.title('Single-cell bacterial growth curves', fontsize = 20, y = 1.03)
				plt.grid(False)

				plt.xlim([0, 60])
				plt.xticks([0,20,40,60],  fontsize = 16)
				plt.ylim([0, 5])
				plt.yticks([0,1,2,3,4,5],  fontsize = 16)
				plt.tight_layout()
				counter += 1

plt.savefig("growthcurve.pdf")
plt.show()


Now lets make a histogram of the instantaneous growth rate. 

In [7]:
counter = 0

cell_size_increases = []
num_bad_segs = 0
num_segs = 0
for tracks in list_of_tracks:
	print 'Chunk number ' + str(counter)
	counter += 1

	for track in tracks:
		if len(track) > 1:
			for j in xrange(len(track)-1):
				cell_size_increase = 0.2 * np.float32((track[j+1]['area'] - track[j]['area'])) *(0.13 **2)
				# if cell_size_increase > 0:
				num_segs += 1
				cell_size_increases += [cell_size_increase]
				if np.float32((track[j+1]['area'] - track[j]['area']))/np.float32(track[j]['area']) > 0.5:
					num_bad_segs += 1

cell_size_increases = np.array(cell_size_increases)
print np.mean(cell_size_increases)
print num_bad_segs
print num_segs
cell_size = plt.hist(cell_size_increases, bins = 470, color = 'r')
plt.xlabel(r'Growth rate ($\mu m^2 / min$)', fontsize = 16)
plt.ylabel(r'Number of cells', fontsize = 16)
plt.title('Instantaneous growth rate', y = 1.03, fontsize = 20)

plt.xlim([-.1, 0.2])
plt.xticks([-.1, 0,0.1,0.2],  fontsize = 16)
plt.ylim([0, 1000])
plt.yticks([0,500,1000],  fontsize = 16)
plt.tight_layout()
plt.savefig("instantaneous_growth_full.pdf")
plt.show()

Chunk number 0
0.00405484641638
1
586


We can also color images by the bacterial growth rate.

In [26]:
growth = np.zeros((870,870,31), dtype = 'float32')
growth_color = 0.2*np.ones((870,870,3,31), dtype = 'float32')

for tracks in list_of_tracks:
    for track in tracks:
        print len(track), track[0]['frame']
        for j in xrange(len(track)):
            cell = track[j]
            if j > 0:
                cell_prev = track[j-1]
            coords_x = cell['coords'][:,0]
            coords_y = cell['coords'][:,1]
            frame_id = cell['frame']
            
            if j > 0:
                growth_rate = 0.2 * np.float32((cell['area'] - cellprev['area'])) *(0.13 **2) *1/0.05
                if np.float32((cell['area'] - cellprev['area'])) > 0.5 * np.float32(cellprev['area']):
                    growth_rate = 0
            else:
                growth_rate = 0
            growth_color[coords_x, coords_y,0,frame_id] = plt.cm.coolwarm(growth_rate)[0]
            growth_color[coords_x, coords_y,1,frame_id] = plt.cm.coolwarm(growth_rate)[1]
            growth_color[coords_x, coords_y,2,frame_id] = plt.cm.coolwarm(growth_rate)[2]
            
            growth[coords_x,coords_y,frame_id] = growth_rate + 10
for j in xrange(30):
    im_name = os.path.join(direc_name, 'growth_color', 'g' + '_' + str(j) + r'.tif')
    tiff.imsave(im_name,np.float32(growth[:,:,j]))

8 1
18 1
30 1
1 1
30 1
5 1
3 1
25 1
4 2
4 2
1 5
19 6
24 6
25 6
21 6
1 6
3 7
20 10
21 10
20 11
19 12
6 12
16 12
9 14
13 14
16 15
2 16
14 17
14 17
14 17
12 17
7 17
13 18
1 18
5 19
7 19
1 19
10 21
10 21
3 22
3 22
7 23
2 23
8 23
8 23
1 23
4 24
7 24
7 24
2 24
7 24
7 24
6 24
1 24
1 24
7 24
7 24
1 25
6 25
2 25
6 25
6 25
6 25
6 25
1 25
6 25
1 26
4 26
5 26
1 26
5 26
5 26
4 27
1 27
1 27
3 27
4 27
4 27
4 27
4 27
1 28
3 28
3 28
3 28
3 28
3 28
3 28
1 29
2 29
2 29
2 29
2 29
2 29
2 29
2 29
2 29
1 30
1 30
1 30
1 30
1 30
1 30
1 30


In [9]:
for k in xrange(1,31):
	print 'Processing image ' + str(k) 
	growth_color = 0.2*np.ones((870,870,3), dtype = 'float32')
	for tracks in list_of_tracks:
		for track in tracks:
			if len(track)>0:
				for j in xrange(1,len(track)):
					cell = track[j]
					cellprev = track[j-1]
					cellzero = track[0]
					frame_id = cell['frame']
					frame_id_zero = cellzero['frame']
					if frame_id == k:
						growth_rate = 0.2 * np.float32((cell['area'] - cellprev['area'])) *(0.13 **2) *1/0.05
						if np.float32((cell['area'] - cellprev['area'])) > 0.5 * np.float32(cellprev['area']):
							growth_rate = 0
						coords_x = cell['coords'][:,0]
						coords_y = cell['coords'][:,1]
						growth_color[coords_x, coords_y,0] = plt.cm.coolwarm(growth_rate)[0]
						growth_color[coords_x, coords_y,1] = plt.cm.coolwarm(growth_rate)[1]
						growth_color[coords_x, coords_y,2] = plt.cm.coolwarm(growth_rate)[2]
						
					if frame_id_zero == k:
						growth_rate = 0
						coords_x = cellzero['coords'][:,0]
						coords_y = cellzero['coords'][:,1]
						growth_color[coords_x, coords_y,0] = plt.cm.coolwarm(growth_rate)[0]
						growth_color[coords_x, coords_y,1] = plt.cm.coolwarm(growth_rate)[1]
						growth_color[coords_x, coords_y,2] = plt.cm.coolwarm(growth_rate)[2]

		im_name = os.path.join(direc_name, 'growth_color', 'growth_color' + '_' + str(k) + r'.tif')
		scipy.misc.imsave(im_name,np.float16(growth_color))

Processing image 1
Processing image 2
Processing image 3
Processing image 4
Processing image 5
Processing image 6
Processing image 7
Processing image 8
Processing image 9
Processing image 10
Processing image 11
Processing image 12
Processing image 13
Processing image 14
Processing image 15
Processing image 16
Processing image 17
Processing image 18
Processing image 19
Processing image 20
Processing image 21
Processing image 22
Processing image 23
Processing image 24
Processing image 25
Processing image 26
Processing image 27
Processing image 28
Processing image 29
Processing image 30
